# Get bert score from best HF autotrain extractive QA model

Assets

In [ ]:
from askem.data import get_covid_qa
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

dataset = get_covid_qa(split='test')
tokenizer = AutoTokenizer.from_pretrained("mbialo/autotrain-test-58072133169")
model = AutoModelForQuestionAnswering.from_pretrained("mbialo/autotrain-test-58072133169")

Inference pipeline

In [ ]:
best_auto_model = pipeline(model=model, tokenizer=tokenizer, task="question-answering", device=2)

def generate_answer(batch) -> dict:
    y = best_auto_model(question=batch['question'], context=batch['context'])
    batch['y_pred'] = [y_['answer'] for y_ in y]
    batch['y_true'] = [a['text'][0] for a in batch['answers']]
    return batch

results = dataset.map(generate_answer, batched=True, batch_size=128)

Calculate BERTScore

In [ ]:
from bert_score import score
precision, recall, f1 = score(results['y_true'], results['y_pred'], lang="en", verbose=True)

Save results to disk

In [ ]:
dataset = dataset.add_column('precision', precision.numpy())
dataset = dataset.add_column('recall', recall.numpy())
dataset = dataset.add_column('f1', f1.numpy())
dataset.to_parquet('/askem/data/bertqa_finetuned.parquet')

Load results

In [ ]:
import pandas as pd
df = pd.read_parquet('/askem/data/bertqa_finetuned.parquet')